In [108]:
from __future__ import division
import numpy as np
import pandas as pd
import math as math
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.gaussian_process.kernels import WhiteKernel
from sklearn.model_selection import train_test_split
from sklearn.gaussian_process.kernels import Matern, RationalQuadratic,ExpSineSquared,PairwiseKernel
from tqdm import tqdm
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from scipy.stats import pearsonr
from sklearn.model_selection import KFold
from sklearn.gaussian_process.kernels import (StationaryKernelMixin,NormalizedKernelMixin,Kernel,Hyperparameter,squareform,pdist,cdist)
import scipy.spatial.distance as distance
import joblib

In [109]:
columns = ["x","y","z","w","data","error"]
datatab = pd.read_table(r"C:\Users\seanw\OneDrive\Desktop\GitHub\FYP\Data\Updateddata.txt",names = columns)
datatab
x = datatab['x']
y = datatab['y']
z = datatab['z']
w = datatab['w']
data = datatab['data']
error = datatab['error']

# First Mass
datam1 = datatab[0:250]
xm1 = datam1['x'].values
ym1 = datam1['y'].values
zm1 = datam1['z'].values
wm1 = datam1['w'].values
dm1 = datam1['data'].values
em1 = datam1['error'].values
X1 = np.column_stack((xm1, ym1, zm1, wm1))

# Second Mass
datam2 = datatab[250:500]
xm2 = datam2['x'].values
ym2 = datam2['y'].values
zm2 = datam2['z'].values
wm2 = datam2['w'].values
dm2 = datam2['data'].values
em2 = datam2['error'].values
X2 = np.column_stack((xm2, ym2, zm2, wm2))

# Third Mass
datam3 = datatab[500:750]
xm3 = datam3['x'].values
ym3 = datam3['y'].values
zm3 = datam3['z'].values
wm3 = datam3['w'].values
dm3 = datam3['data'].values
em3 = datam3['error'].values
X3 = np.column_stack((xm3, ym3, zm3, wm3))

# Fourth Mass
datam4 = datatab[750:]
xm4 = datam4['x'].values
ym4 = datam4['y'].values
zm4 = datam4['z'].values
wm4 = datam4['w'].values
dm4 = datam4['data'].values
em4 = datam4['error'].values
X4 = np.column_stack((xm4, ym4, zm4, wm4))

X = np.column_stack((x,y, z, w))
D = np.concatenate((dm1,dm2,dm3,dm4))
E = np.concatenate((em1,em2,em3,em4))

In [110]:
class RationalQuadraticARD(StationaryKernelMixin, NormalizedKernelMixin, Kernel):
    """
    Rational Quadratic kernel with ARD
    
    lengths: ndarray of shape (n_features>1,)
             list of characteristic lengths, one per input feature.
             
             Suppose the training data is named train_data_X. Then 
             
             lengths = np.ones(np.shape(train_data_X)[1])
    
             would be a suitable input.
             
    Please note: You have to :
        
        import numpy as np
        
        import scipy.spatial.distance as distance
        
        from sklearn.gaussian_process.kernels import Hyperparameter, squareform, pdist, cdist
        
    see formula for RQArd at:
        
        https://stor-i.github.io/GaussianProcesses.jl/latest/kernels/#GaussianProcesses.RQArd
    """
    def __init__(self, lengths: np.ndarray, alpha=1.0, 
                 lengths_bounds=(1e-4, 5), alpha_bounds=(1e-2, 10)):
        self.lengths = lengths              
        self.alpha = alpha
        self.lengths_bounds = lengths_bounds
        self.alpha_bounds = alpha_bounds

    @property
    def hyperparameter_lengths(self):
        return Hyperparameter("lengths", "numeric", self.lengths_bounds, n_elements=len(self.lengths))

    @property
    def hyperparameter_alpha(self):
        return Hyperparameter("alpha", "numeric", self.alpha_bounds)

    def __call__(self, X, Y=None, eval_gradient=False):
        """Return the kernel k(X, Y) and optionally its gradient.
        Parameters
        ----------
        X : ndarray of shape (n_samples_X, n_features)
            Left argument of the returned kernel k(X, Y)
        Y : ndarray of shape (n_samples_Y, n_features), default=None
            Right argument of the returned kernel k(X, Y). If None, k(X, X)
            if evaluated instead.
        eval_gradient : bool, default=False
            Determines whether the gradient with respect to the log of
            the kernel hyperparameter is computed.
            Only supported when Y is None.
        Returns
        -------
        K : ndarray of shape (n_samples_X, n_samples_Y)
            Kernel k(X, Y)
        K_gradient : ndarray of shape (n_samples_X, n_samples_X, n_dims)
            The gradient of the kernel k(X, X) with respect to the log of the
            hyperparameter of the kernel. Only returned when eval_gradient
            is True.
        """
        weights = self.lengths**(-2)
        def mymetric(u, v):
            return distance.sqeuclidean(u, v, w=weights)
            
        X = np.atleast_2d(X)
        if Y is None:
            dists = squareform(pdist(X, metric=mymetric))  # dists is a mxm matrix, where m is the number of datapoints (rows) in X
            tmp = dists / (2 * self.alpha)
            base = (1 + tmp)
            K = (base ** -self.alpha) 
        else:
            if eval_gradient:
                raise ValueError(
                    "Gradient can only be evaluated when Y is None.")
            dists = cdist(X, Y, metric=mymetric)
            K = ((1 + dists / (2 * self.alpha)) ** -self.alpha)

        if eval_gradient:
            # gradient with respect to alpha
            if not self.hyperparameter_alpha.fixed:
                alpha_gradient = \
                     K * (-self.alpha * np.log(base) + dists / (2 * base))
                alpha_gradient = alpha_gradient[:, :, np.newaxis]  
            else:  # alpha is kept fixed
                alpha_gradient = np.empty((K.shape[0], K.shape[1], 0))
   
            # gradient with respect to lengths
            lengths_gradient = []
            for i in range(len(self.lengths)):
                if not self.hyperparameter_lengths.fixed:
                    X_i = X[:,i].reshape(-1,1)
                    dists_i = squareform(pdist(X_i, metric='sqeuclidean'))           
                    length_i_gradient = \
                        dists_i * K / (self.lengths[i] ** 2 * base)
                    length_i_gradient = length_i_gradient[:, :, np.newaxis] 
                    lengths_gradient.append(length_i_gradient)
                else:  # l is kept fixed
                    length_i_gradient = np.empty((K.shape[0], K.shape[1], 0))
                    lengths_gradient.append(length_i_gradient)    
            lengths_gradient = np.swapaxes(lengths_gradient, 0, 3)[0]
            #print(np.mean(np.dstack((alpha_gradient, lengths_gradient, overall_scale_gradient)), axis=(0,1)))
            #print(self.theta)
            return K, np.dstack((alpha_gradient, lengths_gradient))
        else:
            return K

    def __repr__(self):
        return "{0}(alpha={1:.3g}, lengths={2})".format(
            self.__class__.__name__, self.alpha, self.lengths)

### Using Cross Validation to shortlist the best models
- Divide data into 90-10 train-test split
- Use 10-fold cross validation on the train set to shortlist the best models
- I will shortlist the models based on 6 metrics which are RMSE, MAE, R², Adjusted R², Figure of Merit and Pearson Coefficient
- After shortlisting I will build the final model on the entire train set and evaluate it on the test set
- I will also build MCMC models using each of the model types found in the shortlisting step
- I will also compare the MCMC models with the point estimate models using the same metrics

In [111]:
indices = np.arange(X.shape[0])
X_trainval, X_test, D_trainval, D_test, E_trainval, E_test, indices_trainval, indices_test = train_test_split(X, D,E,indices, test_size=0.1, random_state=42)
folds = 10

In [112]:
def build_gpr(set_kernel, set_noise_type, E_train=None):
    """
    Builds a GaussianProcessRegressor given a kernel and noise configuration.
    
    Parameters:
    -----------
    set_kernel : kernel
        A scikit-learn kernel object, e.g. RBF(), Matern(), etc.
    set_noise_type  : str
        A code describing how we handle noise:
          - 'fixed_alpha': use alpha = E_train^2 (no WhiteKernel)
          - 'white_no_error': WhiteKernel with some default noise level
          - 'white_mean_error': WhiteKernel seeded by E_train.mean()
          - 'white_minmax_error': WhiteKernel with bounds from min/max
    E_train : ndarray or float, optional
        If you need the training errors for alpha or WhiteKernel initialization.
    Returns:
    --------
    gpr : GaussianProcessRegressor
        The constructed GPR model with the requested noise approach.
    """

    # Set defaults
    normalize_y = True
    n_restarts_optimizer = 10  
    random_state = 42

    if set_noise_type == 'fixed_alpha':
        if E_train is None:
            raise ValueError("E_train must be provided for 'fixed_alpha' noise config.")
        alpha_val = (E_train)**2
        return GaussianProcessRegressor(kernel=set_kernel, alpha=alpha_val, normalize_y=normalize_y, n_restarts_optimizer=n_restarts_optimizer, random_state=random_state)
    
    elif set_noise_type == 'white_no_error':
        noise_init = 1
        noise_bounds = (1e-6, 1e6)
        combined_kernel = set_kernel + WhiteKernel(noise_level=noise_init, noise_level_bounds=noise_bounds)
        return GaussianProcessRegressor(kernel=combined_kernel, normalize_y=normalize_y, n_restarts_optimizer=n_restarts_optimizer, random_state=random_state)
    
    elif set_noise_type == 'white_mean_error':
        if E_train is None:
            raise ValueError("E_train must be provided for 'white_mean_error'.")
        E_var = E_train**2
        ##### scaling the mean by delta 0.3 for optimising range. Idea  - Keep tight bounds for noise
        noise_init = np.mean(E_var)
        noise_bounds = (np.mean(E_var)*0.7, np.mean(E_var)*1.3)
        combined_kernel = set_kernel + WhiteKernel(noise_level=noise_init, noise_level_bounds=noise_bounds)
        return GaussianProcessRegressor(kernel=combined_kernel, normalize_y=normalize_y, n_restarts_optimizer=n_restarts_optimizer, random_state=random_state)
    
    elif set_noise_type == 'white_minmax_error':
        if E_train is None:
            raise ValueError("E_train must be provided for 'white_minmax_error'.")
        E_var = E_train**2
        ##### bounds for noise level decided by the range(5,95)% of error. Tighter than no_error but not as tight as mean. Allowing for the model to have a decent bit of variation
        p5, p95 = np.percentile(E_var, [5, 95])
        noise_init = np.mean(E_var)
        noise_bounds = (p5, p95)
        combined_kernel = set_kernel + WhiteKernel(noise_level=noise_init, noise_level_bounds=noise_bounds)
        return GaussianProcessRegressor(kernel=combined_kernel, normalize_y=normalize_y, n_restarts_optimizer=n_restarts_optimizer,random_state=random_state)
    
    elif set_noise_type == "hybrid":
        if E_train is None:
            raise ValueError("E_train must be provided for 'fixed_alpha' noise config.")
        alpha_val = (E_train)**2
        noise_init = 0.1
        noise_bounds = (1e-3,0.5)
        combined_kernel = set_kernel + WhiteKernel(noise_level=noise_init, noise_level_bounds=noise_bounds)
        return GaussianProcessRegressor(kernel=combined_kernel, alpha = alpha_val, normalize_y=normalize_y,n_restarts_optimizer=n_restarts_optimizer,random_state=random_state)
    
    elif set_noise_type == "combined_kernel":
        return GaussianProcessRegressor(kernel=set_kernel, normalize_y=normalize_y, n_restarts_optimizer=n_restarts_optimizer,random_state=random_state)
    else:
        raise ValueError("Invalid noise type.")

In [113]:
def MCSGPR(kernel,X_train,D_train,E_train,Xpred,X_test,D_test,n_samples,n_gp_samples):
    posterior_history = []  
    posterior_history_test = [] 
    for sample in tqdm(range(n_samples), desc="Monte Carlo Sampling Progress"):

        ####  This follows the logic since we are saying the error is normally distributed with error 0 and std E_train. There for the error bars which represent 1 standard deviation 
        sampled_D = D_train + np.random.normal(0, E_train)

        gpr = GaussianProcessRegressor(kernel=kernel, normalize_y=True,n_restarts_optimizer = 5,random_state=42)
        gpr.fit(X_train, sampled_D)

        posterior_sample = gpr.sample_y(Xpred, n_gp_samples)
        posterior_sample_test = gpr.sample_y(X_test, n_gp_samples)
        posterior_history.append(posterior_sample)  
        posterior_history_test.append(posterior_sample_test)  
 

    
    
    posterior_history = np.array(posterior_history)  
    posterior_history_test = np.array(posterior_history_test) 
    #### axis = (0,2) if more than 1 gp_sample
    final_mean = np.mean(posterior_history,axis=(0,2))
    final_std = np.std(posterior_history,axis=(0,2))
    final_mean_test = np.mean(posterior_history_test,axis=(0,2))
    final_std_test = np.std(posterior_history_test,axis=(0,2))
    rmse = np.sqrt(mean_squared_error(D_test, final_mean_test))
    mae = mean_absolute_error(D_test, final_mean_test)
    r2 = r2_score(D_test, final_mean_test)
    adjusted_r2 = 1 - (1-r2)*(len(D_test)-1)/(len(D_test)-X_test.shape[1]-1)
    fom = np.mean(np.abs(D_test - final_mean_test)/final_std_test)
    pearsoncoeff, p_value = pearsonr(D_test.ravel(), final_mean_test.ravel())

    return final_mean, final_std,final_mean_test,final_std_test,rmse,mae,r2,adjusted_r2,fom,pearsoncoeff

In [114]:
def cross_validate_gpr(kernel, modeltypes, X_trainval, D_trainval, E_trainval, folds=10,mc_samples=10, gp_samples=5):
    """
    Runs cross-validation for a given kernel and list of model types (noise types).
    
    Parameters:
    -----------
    kernel : sklearn kernel object
        The base kernel (e.g., RBF(), RationalQuadratic(), Matern()).
    modeltypes : list of str
        List of noise model types (e.g., ['white_no_error', 'white_mean_error', ...]).
    X_trainval, D_trainval, E_trainval : ndarray
        Training/validation data and associated error.
    folds : int
        Number of folds (default 10).
    mc_samples : int
        Number of Monte Carlo samples (for 'monte_carlo' modeltype).
    gp_samples : int
        Number of function samples from GP posterior (for 'monte_carlo' modeltype).
    
    Returns:
    --------
    cv_results : dict
        Dictionary with metrics for each model type.
    """

    cv_results = {}
    kf = KFold(n_splits=folds, shuffle=True, random_state=42)

    for modeltype in modeltypes:
        print(f"\nRunning CV for model type: {modeltype}")

        r2, rmse, mae, adjusted_r2, fom, pearsoncoeff = [], [], [], [], [], []

        for train_index, val_index in tqdm(list(kf.split(X_trainval)), total=folds, desc=f"CV {modeltype}"):
            X_train, X_val = X_trainval[train_index], X_trainval[val_index]
            D_train, D_val = D_trainval[train_index], D_trainval[val_index]
            E_train, _ = E_trainval[train_index], E_trainval[val_index]

            if modeltype == "combined_kernel":
                gpr = build_gpr(kernel,modeltype,E_train)
                gpr.fit(X_train, D_train)
                D_pred, D_std = gpr.predict(X_val, return_std=True)

                rmse_val = np.sqrt(mean_squared_error(D_val, D_pred))
                mae_val = mean_absolute_error(D_val, D_pred)
                r2_val = r2_score(D_val, D_pred)
                adj_r2_val = 1 - (1 - r2_val) * (len(D_val) - 1) / (len(D_val) - X_val.shape[1] - 1)
                fom_val = np.mean(np.abs(D_val - D_pred) / D_std)
                pearson_val, _ = pearsonr(D_val.ravel(), D_pred.ravel())

            elif modeltype == "monte_carlo":
                _, _, _, _, rmse_val, mae_val, r2_val, adj_r2_val, fom_val, pearson_val = MCSGPR(kernel, X_train, D_train, E_train, X_val, X_val, D_val,
                    n_samples=mc_samples, n_gp_samples=gp_samples)
                
            else:
                gpr = build_gpr(kernel, modeltype, E_train)
                gpr.fit(X_train, D_train)
                D_pred, D_std = gpr.predict(X_val, return_std=True)

                rmse_val = np.sqrt(mean_squared_error(D_val, D_pred))
                mae_val = mean_absolute_error(D_val, D_pred)
                r2_val = r2_score(D_val, D_pred)
                adj_r2_val = 1 - (1 - r2_val) * (len(D_val) - 1) / (len(D_val) - X_val.shape[1] - 1)
                fom_val = np.mean(np.abs(D_val - D_pred) / D_std)
                pearson_val, _ = pearsonr(D_val.ravel(), D_pred.ravel())

            # Append to metrics lists
            rmse.append(rmse_val)
            mae.append(mae_val)
            r2.append(r2_val)
            adjusted_r2.append(adj_r2_val)
            fom.append(fom_val)
            pearsoncoeff.append(pearson_val)

        # Store metrics for this modeltype
        cv_results[modeltype] = {'r2': r2,'rmse': rmse,'mae': mae,'adjusted_r2': adjusted_r2,'fom': fom,'pearson': pearsoncoeff}

    return cv_results


In [115]:
bounds_lmult = ((1e-4, 10), (1e-4, 10.0), (1e-4, 10.0), (1e-4, 10.0))  # Fix l1 bounds
guess_lmult = (1,1,1,1)
bounds_lsingle = (1e-4, 10.0)
ExpSinelbounds = (1e-2, 10.0)
guess_lsingle = 1.0
guess_signal_var = 1.0
bounds_signal_var = (1e-2, 20)
nu = 1.75
nu2 = 0.75
alpha = 1
periodicity = 2
periodictybounds = (1, 20)
gamma = 1
Matkernel = C(constant_value=guess_signal_var, constant_value_bounds=bounds_signal_var) * Matern(length_scale=guess_lmult, length_scale_bounds=bounds_lmult, nu=nu)
Radkernel = C(constant_value=guess_signal_var, constant_value_bounds=bounds_signal_var) *RationalQuadratic(length_scale=guess_lsingle, alpha=alpha, length_scale_bounds=bounds_lsingle,alpha_bounds=(0.1,50))   ####RationalQuadraticARD(lengths=np.array([1,1,1,1]))
ExpSinekernel = C(constant_value=guess_signal_var, constant_value_bounds=bounds_signal_var) * ExpSineSquared(length_scale=guess_lsingle, periodicity=periodicity, length_scale_bounds=ExpSinelbounds)
Laplacekernel = C(constant_value=guess_signal_var, constant_value_bounds=bounds_signal_var) * PairwiseKernel(gamma=gamma, gamma_bounds=(0.1,50), metric='laplacian')
RBFkernel = C(constant_value=guess_signal_var, constant_value_bounds=bounds_signal_var) * RBF(length_scale=guess_lmult, length_scale_bounds=bounds_lmult)




kernels = {"Matern": Matkernel,"RationalQuadratic": Radkernel,"ExpSineSquared": ExpSinekernel,"Laplace": Laplacekernel,"RBF": RBFkernel}

modeltypes = ['white_no_error', 'white_mean_error','white_minmax_error','fixed_alpha','hybrid','monte_carlo']

cv_results_all_kernels = {}
for kernel_name, kernel_obj in kernels.items():
        print(f"\n\nRunning CV for kernel: {kernel_name}")
        cv_results = cross_validate_gpr(kernel_obj, modeltypes, X_trainval, D_trainval, E_trainval, folds=folds)
        # Store results in the final dictionary under kernel name
        cv_results_all_kernels[kernel_name] = cv_results



Running CV for kernel: Matern

Running CV for model type: white_no_error


CV white_no_error: 100%|██████████| 10/10 [22:31<00:00, 135.11s/it]



Running CV for model type: white_mean_error


CV white_mean_error:   0%|          | 0/10 [00:00<?, ?it/s]c:\Users\seanw\anaconda3\Lib\site-packages\sklearn\gaussian_process\kernels.py:445: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified lower bound 0.5143804386540594. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
CV white_mean_error:  10%|█         | 1/10 [01:38<14:43, 98.18s/it]c:\Users\seanw\anaconda3\Lib\site-packages\sklearn\gaussian_process\kernels.py:445: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified lower bound 0.5267006904199005. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
CV white_mean_error:  20%|██        | 2/10 [03:16<13:05, 98.24s/it]c:\Users\seanw\anaconda3\Lib\site-packages\sklearn\gaussian_process\kernels.py:445: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is


Running CV for model type: white_minmax_error


...

CV fixed_alpha: 100%|██████████| 10/10 [03:34<00:00, 21.42s/it]



Running CV for model type: hybrid


CV hybrid:   0%|          | 0/10 [00:00<?, ?it/s]c:\Users\seanw\anaconda3\Lib\site-packages\sklearn\gaussian_process\kernels.py:445: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified lower bound 0.001. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
CV hybrid:  10%|█         | 1/10 [00:38<05:44, 38.24s/it]c:\Users\seanw\anaconda3\Lib\site-packages\sklearn\gaussian_process\kernels.py:445: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified lower bound 0.001. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
CV hybrid:  20%|██        | 2/10 [01:18<05:17, 39.69s/it]c:\Users\seanw\anaconda3\Lib\site-packages\sklearn\gaussian_process\kernels.py:445: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified lower bound 0.001. Decreasing th


Running CV for model type: monte_carlo


CV monte_carlo:   0%|          | 0/10 [00:00<?, ?it/s]c:\Users\seanw\anaconda3\Lib\site-packages\sklearn\gaussian_process\kernels.py:445: ConvergenceWarning: The optimal value found for dimension 1 of parameter k2__length_scale is close to the specified lower bound 0.0001. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
c:\Users\seanw\anaconda3\Lib\site-packages\sklearn\gaussian_process\kernels.py:445: ConvergenceWarning: The optimal value found for dimension 2 of parameter k2__length_scale is close to the specified lower bound 0.0001. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
c:\Users\seanw\anaconda3\Lib\site-packages\sklearn\gaussian_process\kernels.py:445: ConvergenceWarning: The optimal value found for dimension 3 of parameter k2__length_scale is close to the specified lower bound 0.0001. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
CV monte_carlo:  10%|█         

In [116]:
cv_results_all_kernels
for kernel_name, kernel_obj in kernels.items():
    savepath = save_path = f"cv_results_{kernel_name}.pkl"
    cv_results = cv_results_all_kernels[kernel_name]
    joblib.dump(cv_results, savepath)

In [117]:
guess_signal_var = 1.0
bounds_signal_var = (1e-2, 20)

RBFMatkernel = C(constant_value=guess_signal_var, constant_value_bounds=bounds_signal_var) * RBF(length_scale=(1,1,1,1), length_scale_bounds=(0.5,100)) + C(constant_value=guess_signal_var, constant_value_bounds=bounds_signal_var) * Matern(length_scale=(0.1,0.1,0.1,0.1), length_scale_bounds=(1e-6,4), nu=0.75)
RBFRadkernel = C(constant_value=guess_signal_var, constant_value_bounds=bounds_signal_var) * RBF(length_scale=(1,1,1,1), length_scale_bounds=(0.5,100)) + C(constant_value=guess_signal_var, constant_value_bounds=bounds_signal_var) * RationalQuadratic(length_scale=guess_lsingle, alpha=alpha, length_scale_bounds=bounds_lsingle,alpha_bounds=(0.01,0.5))   #####RationalQuadraticARD(lengths=np.array([0.1,0.1,0.1,0.1]), alpha = 0.2,lengths_bounds=(1e-6,4),alpha_bounds=(0.01,0.5))
RBFLaplacekernel = C(constant_value=guess_signal_var, constant_value_bounds=bounds_signal_var) * RBF(length_scale=(1,1,1,1), length_scale_bounds=(0.5,100)) + C(constant_value=guess_signal_var, constant_value_bounds=bounds_signal_var) * PairwiseKernel(gamma=1, gamma_bounds = (0.5,5),metric='laplacian')


combinedkernels = {"RBFMatkernel":RBFMatkernel,"RBFRadkernel":RBFRadkernel,"RBFLaplacekernel":RBFLaplacekernel}

cv_results_combinedkernel = {}

for kernel_name, kernel_obj in combinedkernels.items():
    modeltype = "combined_kernel"
    print(f"\n\nRunning CV for kernel: {kernel_name}")
    cv_results = cross_validate_gpr(kernel_obj, [modeltype], X_trainval, D_trainval, E_trainval, folds=folds)
    cv_results_combinedkernel[kernel_name] = cv_results



Running CV for kernel: RBFMatkernel

Running CV for model type: combined_kernel


CV combined_kernel:   0%|          | 0/10 [00:00<?, ?it/s]c:\Users\seanw\anaconda3\Lib\site-packages\sklearn\gaussian_process\kernels.py:445: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__k2__length_scale is close to the specified lower bound 0.5. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
c:\Users\seanw\anaconda3\Lib\site-packages\sklearn\gaussian_process\kernels.py:445: ConvergenceWarning: The optimal value found for dimension 3 of parameter k1__k2__length_scale is close to the specified lower bound 0.5. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
CV combined_kernel:  10%|█         | 1/10 [03:15<29:22, 195.88s/it]c:\Users\seanw\anaconda3\Lib\site-packages\sklearn\gaussian_process\kernels.py:445: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__k2__length_scale is close to the specified lower bound 0.5. Decreasing the bound and calling fit again



Running CV for kernel: RBFRadkernel

Running CV for model type: combined_kernel


CV combined_kernel:   0%|          | 0/10 [00:00<?, ?it/s]c:\Users\seanw\anaconda3\Lib\site-packages\sklearn\gaussian_process\kernels.py:445: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__k2__length_scale is close to the specified lower bound 0.5. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
c:\Users\seanw\anaconda3\Lib\site-packages\sklearn\gaussian_process\kernels.py:455: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__k2__alpha is close to the specified upper bound 0.5. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
c:\Users\seanw\anaconda3\Lib\site-packages\sklearn\gaussian_process\kernels.py:445: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__k2__length_scale is close to the specified lower bound 0.0001. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
CV combined_kernel:  10%|█  



Running CV for kernel: RBFLaplacekernel

Running CV for model type: combined_kernel


CV combined_kernel:   0%|          | 0/10 [00:00<?, ?it/s]c:\Users\seanw\anaconda3\Lib\site-packages\sklearn\gaussian_process\kernels.py:445: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__k2__length_scale is close to the specified lower bound 0.5. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
c:\Users\seanw\anaconda3\Lib\site-packages\sklearn\gaussian_process\kernels.py:445: ConvergenceWarning: The optimal value found for dimension 1 of parameter k1__k2__length_scale is close to the specified lower bound 0.5. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
c:\Users\seanw\anaconda3\Lib\site-packages\sklearn\gaussian_process\kernels.py:445: ConvergenceWarning: The optimal value found for dimension 2 of parameter k1__k2__length_scale is close to the specified lower bound 0.5. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
c:\Users\seanw\anaconda3

In [118]:
cv_results_combinedkernel
for kernel_name, kernel_obj in combinedkernels.items():
    savepath = save_path = f"cv_results_{kernel_name}.pkl"
    cv_results = cv_results_combinedkernel[kernel_name]
    joblib.dump(cv_results, savepath)
##joblib.dump(cv_results_combinedkernel, 'cv_results_combinedkernel.pkl')